In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
df=pd.read_csv("/kaggle/input/gestational-diabetes/Gestational Diabetes.csv")

In [4]:
df.isnull().sum()

Age             0
Pregnancy No    0
Weight          0
Height          0
BMI             0
Heredity        0
Prediction      0
dtype: int64

In [5]:
X=df.drop(columns=["Prediction"])
Y=df["Prediction"]

In [15]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1,random_state=42)

In [19]:
random_class=RandomForestClassifier(n_estimators=50)
random_class.fit(X_train,Y_train)
random_class.score(X_test,Y_test)

0.8627450980392157

In [20]:
x_train=X_train

In [22]:
!pip install cirq tensorflow_quantum

INFO: pip is looking at multiple versions of tensorflow-quantum to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 5.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of qcs-api-client to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of qcs-api-client to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15

In [26]:
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import sympy 
import numpy as np

In [48]:
x_train.shape[1]

6

In [50]:
def create_quantum_circuits(x_train):
    qubits = [cirq.GridQubit(0, i) for i in range(x_train.shape[-1])]
    circuits = []
#     for x in x_train:
    circuit = cirq.Circuit()
    for i, value in enumerate(x_train):
         if value:
            circuit.append(cirq.X(qubits[i]))
    circuits.append(circuit)
    return circuits, qubits

circuits, qubits = create_quantum_circuits(x_train)
def create_model(qubits):
    # Define symbolic parameters
    symbols = sympy.symbols(f'theta0:{len(qubits)}')

    # Create parameterized circuit
    circuit = cirq.Circuit()
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.rx(symbols[i])(qubit))
    
    readout_operators = [cirq.Z(qubit) for qubit in qubits]
    
    inputs = tf.keras.Input(shape=(x_train.shape[-1],), dtype=tf.dtypes.string)
    pqc = tfq.layers.PQC(circuit, readout_operators)(inputs)
    
    return tf.keras.Model(inputs=inputs, outputs=pqc)

model = create_model(qubits)
# _________________________
quantum_data = tfq.convert_to_tensor(circuits)  # Convert circuits to tensor
def hinge_loss(y_true, y_pred):
    return tf.reduce_mean(tf.maximum(1.0 - y_true * y_pred, 0.0))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=hinge_loss,metrics=['accuracy'])
history = model.fit(quantum_data, y_train, epochs=100, verbose=1)
loss = model.evaluate(quantum_data, y_train)
print('Loss:', loss)

TypeError: Layer.add_weight() got multiple values for argument 'shape'

In [30]:
x_train.shape[0]

910

In [31]:
x_train.shape[1]

6

In [35]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq

def create_model(qubits):
    # Create a parameterized quantum circuit
    circuit = cirq.Circuit()
    for qubit in qubits:
        circuit.append(cirq.H(qubit))
        circuit.append(cirq.X(qubit) ** sympy.Symbol(f'theta_{qubit}'))

    readout_operators = [cirq.Z(qubit) for qubit in qubits]

    # Create the Keras Input layer
    inputs = tf.keras.Input(shape=(), dtype=tf.dtypes.string)

    # Create the PQC layer
    pqc = tfq.layers.PQC(circuit, readout_operators)(inputs)

    return tf.keras.Model(inputs=inputs, outputs=pqc)

# Define your qubits
qubits = [cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)]

# Create the model
model = create_model(qubits)

# Convert your quantum data to tensor (example data)
quantum_data = tfq.convert_to_tensor([circuit for _ in range(10)])  # Adjust the circuits list as per your data


TypeError: Layer.add_weight() got multiple values for argument 'shape'

In [36]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy

def create_model(qubits):
    # Create a parameterized quantum circuit
    circuit = cirq.Circuit()
    symbols = sympy.symbols(f'theta_0:{len(qubits)}')
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.H(qubit))
        circuit.append(cirq.X(qubit) ** symbols[i])

    readout_operators = [cirq.Z(qubit) for qubit in qubits]

    # Create the Keras Input layer
    inputs = tf.keras.Input(shape=(), dtype=tf.dtypes.string)

    # Create the PQC layer
    pqc = tfq.layers.PQC(circuit, readout_operators)(inputs)

    return tf.keras.Model(inputs=inputs, outputs=pqc)

# Define your qubits
qubits = [cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)]

# Create the model
model = create_model(qubits)

# Convert your quantum data to tensor (example data)
circuits = [cirq.Circuit(cirq.H(q) for q in qubits)] * 10  # Example: 10 circuits
quantum_data = tfq.convert_to_tensor(circuits)

# Example usage: model.predict(quantum_data)
print(model.summary())


TypeError: Layer.add_weight() got multiple values for argument 'shape'

In [37]:
pip install tensorflow==2.4.1 tensorflow-quantum==0.4.0 cirq==0.9.1


ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0)
ERROR: No matching distribution found for tensorflow==2.4.1
Note: you may need to restart the kernel to use updated packages.


In [43]:
x_train.shape

(910, 6)

In [44]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy

def create_model(qubits):
    # Define the symbols for the parameterized gates
    symbols = sympy.symbols(f'theta_0:{len(qubits)}')

    # Create a parameterized quantum circuit
    circuit = cirq.Circuit()
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.H(qubit))  # Apply Hadamard gate to each qubit
        circuit.append(cirq.rx(symbols[i])(qubit))  # Apply parameterized RX gate

    # Define readout operators for each qubit
    readout_operators = [cirq.Z(qubit) for qubit in qubits]

    # Create the Keras Input layer
    inputs = tf.keras.Input(shape=(6,), dtype=tf.dtypes.string)

    # Create the PQC layer
    pqc = tfq.layers.PQC(circuit, readout_operators)(inputs)

    # Create and return the Keras model
    return tf.keras.Model(inputs=inputs, outputs=pqc)

# Define your qubits
qubits = [cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)]

# Create the model
model = create_model(qubits)

# Example circuits for conversion (adjust this as per your actual data)
circuits = [cirq.Circuit(cirq.H(q) for q in qubits)] * 10  # Example: 10 circuits

# Convert circuits to quantum tensors
quantum_data = tfq.convert_to_tensor(circuits)

# Example usage: predict with the model
print(model.summary())


TypeError: Layer.add_weight() got multiple values for argument 'shape'

In [39]:
import tensorflow_quantum as tfq

# Print TensorFlow Quantum version
print("TensorFlow Quantum version:", tfq.__version__)


TensorFlow Quantum version: 0.7.2


In [40]:
import tensorflow as tf
import cirq

# Print TensorFlow and Cirq versions
print("TensorFlow version:", tf.__version__)
print("Cirq version:", cirq.__version__)


TensorFlow version: 2.15.0
Cirq version: 1.3.0


In [41]:
import tensorflow_quantum as tfq
import cirq
import sympy
import tensorflow as tf

# Define a qubit
qubit = cirq.GridQubit(0, 0)

# Define a simple quantum circuit
circuit = cirq.Circuit(cirq.H(qubit))

# Convert the circuit to a tensor
quantum_data = tfq.convert_to_tensor([circuit])

# Define a simple model using PQC layer
class SimpleModel(tf.keras.Model):
    def __init__(self, qubit):
        super(SimpleModel, self).__init__()
        self.pqc = tfq.layers.PQC(cirq.Circuit(cirq.H(qubit)), [cirq.Z(qubit)])

    def call(self, inputs):
        return self.pqc(inputs)

# Create and compile the model
model = SimpleModel(qubit)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.MeanSquaredError())

# Print model summary
print(model.summary())

# Perform a simple prediction
output = model.predict(quantum_data)
print("Model output:", output)


ValueError: model_circuit has no sympy.Symbols. Please provide a circuit that contains symbols so that their values can be trained.

In [42]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy

# Define a qubit
qubit = cirq.GridQubit(0, 0)

# Define a parameterized quantum circuit with sympy.Symbols
theta = sympy.Symbol('theta')
circuit = cirq.Circuit(cirq.H(qubit), cirq.rx(theta)(qubit))

# Convert the circuit to a tensor
quantum_data = tfq.convert_to_tensor([circuit])

# Define a simple model using PQC layer
class SimpleModel(tf.keras.Model):
    def __init__(self, qubit):
        super(SimpleModel, self).__init__()
        # Ensure the circuit has parameterized gates using sympy.Symbol
        theta = sympy.Symbol('theta')
        circuit = cirq.Circuit(cirq.H(qubit), cirq.rx(theta)(qubit))
        self.pqc = tfq.layers.PQC(circuit, [cirq.Z(qubit)])

    def call(self, inputs):
        return self.pqc(inputs)

# Create and compile the model
model = SimpleModel(qubit)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.MeanSquaredError())

# Print model summary
print(model.summary())

# Perform a simple prediction
output = model.predict(quantum_data)
print("Model output:", output)


TypeError: Layer.add_weight() got multiple values for argument 'shape'